In [52]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

API_KEY = "bob_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

IPython autoawait is `on`, and set to use `asyncio`


In [53]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 3. connect to SP

In [54]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


Listening webhooks failed! OSError(98, "error while attempting to bind on address ('0.0.0.0', 8052): address already in use") occurred.


In [55]:
#Paste in invitation from Carol agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '082d1357-64ec-4653-84af-777dd838b48f', 'label': 'Carol', 'serviceEndpoint': 'http://192.168.65.3:8080', 'recipientKeys': ['AjzspEpMcf53s9nS4XcBogeH8zKHWkESzbLKbPV1TtKq']}

In [56]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Carol's connection ID
print("Connection", response)
sp_connection = response["connection_id"]


Connection {'their_role': 'inviter', 'invitation_key': 'AjzspEpMcf53s9nS4XcBogeH8zKHWkESzbLKbPV1TtKq', 'their_label': 'Carol', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': '48a27447-45ff-4d41-985e-ae9ca6130d40', 'state': 'request', 'connection_id': '24d83a42-68fa-45e5-96d5-f59481212810', 'created_at': '2023-01-03 11:26:15.088781Z', 'rfc23_state': 'request-sent', 'updated_at': '2023-01-03 11:26:15.147151Z', 'routing_state': 'none', 'my_did': 'GAAHQwGB13skFWsLAgYdRZ'}


In [5]:
# sp_connection = '73437da8-d87e-46b7-aa1e-c1ccc86fccf7'

### switch to SP

In [57]:
# Print connection list
connection = await agent_controller.connections.get_connection(sp_connection)
print("SP AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

SP AGENT CONNECTION
{'their_role': 'inviter', 'invitation_key': 'AjzspEpMcf53s9nS4XcBogeH8zKHWkESzbLKbPV1TtKq', 'their_label': 'Carol', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': '48a27447-45ff-4d41-985e-ae9ca6130d40', 'state': 'active', 'connection_id': '24d83a42-68fa-45e5-96d5-f59481212810', 'created_at': '2023-01-03 11:26:15.088781Z', 'rfc23_state': 'completed', 'updated_at': '2023-01-03 11:26:27.598160Z', 'routing_state': 'none', 'my_did': 'GAAHQwGB13skFWsLAgYdRZ', 'their_did': 'HHG2eTSodgTkGHU2QQUDBY'}
State: active


In [58]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'their_role': 'inviter', 'invitation_key': '8yDBAPXWfTEKdKCdRHBr7rKVpJ5pZkQ2bLGoJEXFbxCX', 'their_label': 'Alice', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': '22d9839a-dded-477d-a7f7-300187999d92', 'state': 'request', 'connection_id': '0380cef4-1ad8-44d1-9d3b-aeb00d16daf2', 'created_at': '2023-01-03 09:28:18.914466Z', 'rfc23_state': 'request-sent', 'updated_at': '2023-01-03 09:28:18.975253Z', 'routing_state': 'none', 'my_did': 'X912oZQ5u5JLBk7fDT2LfT'}, {'their_role': 'inviter', 'invitation_key': '8fzHn56Ws5XGrbnFgFC7cvNxAJaQ5H6twaRWGA8TKphM', 'their_label': 'Alice', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': '3c858a56-d4f3-4f83-91d6-093f66e0ab82', 'state': 'active', 'connection_id': '3fa138e7-7965-48c9-b731-8fc1d72c2d3e', 'created_at': '2023-01-03 09:28:35.129989Z', 'rfc23_state': 'completed', 'updated_at': '2023-01-03 09:29:15.049797Z', 'routing_state': 'none', 'my_did': 'VjHmiqvFUDKtfjzadgFhjk', 'their_did': 'QRT3h2UtVYUFwkjVEJjK5U'}

## 5. connect to DM

In [69]:
#Paste in invitation from Carol agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'ff2a54fa-b5c7-40b5-98f9-1c4fab300b2f', 'label': 'Alice', 'recipientKeys': ['EASvR9t7Dkb8pUta1dDcpZwDJj32zCLkXpjFvPotNTWx'], 'serviceEndpoint': 'http://192.168.65.3:8020'}

In [70]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Carol's connection ID
print("Connection", response)
dm_connection = response["connection_id"]

Connection {'their_role': 'inviter', 'invitation_key': 'EASvR9t7Dkb8pUta1dDcpZwDJj32zCLkXpjFvPotNTWx', 'their_label': 'Alice', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': '9fdc1952-b260-4292-b0d0-e24b5bdd72ba', 'state': 'request', 'connection_id': 'f2416802-9c49-406e-8035-3415108a72ca', 'created_at': '2023-01-03 11:31:24.992814Z', 'rfc23_state': 'request-sent', 'updated_at': '2023-01-03 11:31:25.035068Z', 'routing_state': 'none', 'my_did': 'KBtYiK8eaWNKEtnVWAR64h'}


### switch to DM

In [71]:
connection = await agent_controller.connections.get_connection(dm_connection)
print(connection)
print("Is Active?", connection["state"])

{'their_role': 'inviter', 'invitation_key': 'EASvR9t7Dkb8pUta1dDcpZwDJj32zCLkXpjFvPotNTWx', 'their_label': 'Alice', 'invitation_mode': 'once', 'accept': 'manual', 'request_id': '9fdc1952-b260-4292-b0d0-e24b5bdd72ba', 'state': 'active', 'connection_id': 'f2416802-9c49-406e-8035-3415108a72ca', 'created_at': '2023-01-03 11:31:24.992814Z', 'rfc23_state': 'completed', 'updated_at': '2023-01-03 11:31:58.077211Z', 'routing_state': 'none', 'my_did': 'KBtYiK8eaWNKEtnVWAR64h', 'their_did': 'US6mN2w2jdYCrwiBQdBzQE'}
Is Active? active


## 8. exchange DID with DM

In [72]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
dm_did = 'None'

my DID gZj39Y8JGQ5GVQp2y6zFH


In [73]:
def messages_handler(payload):
    global dm_did
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)
    dm_did = payload['content']
    print('DM DID', dm_did)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

Subscribing too: basicmessages


Listening webhooks failed! OSError(98, "error while attempting to bind on address ('0.0.0.0', 8052): address already in use") occurred.


In [42]:
await agent_controller.messaging.trust_ping(dm_connection, "hello, world!")

{'thread_id': '695118a3-4e08-410a-aa2e-1d265664a30e'}

In [74]:
response = await agent_controller.messaging.send_message(dm_connection, 'music_'+my_did)
print(response)

{}


In [35]:
print(dm_did)

None


## present proof that itself belongs to DM to SP

In [13]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")
          
proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)

Subscribing too: present_proof


Listening webhooks failed! OSError(98, "error while attempting to bind on address ('0.0.0.0', 8052): address already in use") occurred.


### switch to SP

In [14]:
response = await agent_controller.proofs.get_records()
print((response['results']))

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][0]['presentation_exchange_id'])
print('Presentation Request Object\n')
print(response['results'][0]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

[{'trace': False, 'presentation_request': {'name': 'Proof of Ownership of DM', 'version': '1.0', 'requested_attributes': {'0_author_uuid': {'name': 'author', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_origin_uuid': {'name': 'origin', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_owner_uuid': {'name': 'owner', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_time_uuid': {'name': 'time', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'requested_predicates': {}, 'nonce': '857932762503873741699352'}, 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '3264ffcd-fed8-434e-9406-9eacf5ad7d7c', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIE93bmVyc2hpcCBvZiBETSIsICJ2ZXJzaW9uIjogIjEuMCIsICJyZXF1ZXN0ZWRfYXR0cmlidXRlcyI6IHsiMF9hdXRob3JfdXVpZCI6IHsibmFtZ

In [15]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

Received Request -> Query for credentials in the wallet that satisfy the proof request
[{'cred_info': {'referent': 'My OM Credential', 'attrs': {'origin': 'PQRXDxdGqQGSZ8z69p4xZP', 'version': 'v1', 'name': 'balabababa music', 'author': 'PQRXDxdGqQGSZ8z69p4xZP', 'owner': 'PQRXDxdGqQGSZ8z69p4xZP', 'time': '01-03-2023, 09:31:44'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default', 'rev_reg_id': None, 'cred_rev_id': None}, 'interval': None, 'presentation_referents': ['0_author_uuid', '0_time_uuid', '0_origin_uuid', '0_owner_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_author_uuid': {'cred_id': 'My OM Credential', 'revealed': True}, '0_origin_uuid': {'cred_id': 'My OM Credential', 'revealed': True}, '0_owner_uuid': {'cred_id': 'My OM Credential', 'revealed': True}, '0_time_uuid': {'cred_id': 'My OM Credential', 'revealed': True}}, 'self_attested_attributes': {}}

XXX
{}
{'0_author_uuid

In [16]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

{'trace': False, 'presentation_request': {'name': 'Proof of Ownership of DM', 'version': '1.0', 'requested_attributes': {'0_author_uuid': {'name': 'author', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_origin_uuid': {'name': 'origin', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_owner_uuid': {'name': 'owner', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_time_uuid': {'name': 'time', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'requested_predicates': {}, 'nonce': '857932762503873741699352'}, 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'author': '43137799088118443800136745895388480458686925529355594539512585697577946794869', 'origin': '43137799088118443800136745895388480458686925529355594539512585697577946794869', 'owner': '43137799088118443800136745895388480458686925529355594539512585697577946794869', 'time': '754764342976324744712078016003013782185510764418549704993008

## save new VC

In [17]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_credential


Listening webhooks failed! OSError(98, "error while attempting to bind on address ('0.0.0.0', 8052): address already in use") occurred.


In [19]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange d27b4215-d0e5-4151-9897-12c1c38bcb6a, role: holder, state: offer_received
Being offered: [{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': '8HWQruWXrv8s6BBLXm53B'}, {'name': 'time', 'value': '01-03-2023, 09:50:36'}, {'name': 'version', 'value': 'v1'}, {'name': 'name', 'value': 'balabababa music'}]


In [21]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

ClientResponseError: 400, message='Credential exchange d27b4215-d0e5-4151-9897-12c1c38bcb6a in credential_received state (must be offer_received).', url=URL('http://bob-agent:8051/issue-credential/records/d27b4215-d0e5-4151-9897-12c1c38bcb6a/send-request')

Error during POST /issue-credential/records/d27b4215-d0e5-4151-9897-12c1c38bcb6a/send-request: 400, message='Credential exchange d27b4215-d0e5-4151-9897-12c1c38bcb6a in credential_received state (must be offer_received).', url=URL('http://bob-agent:8051/issue-credential/records/d27b4215-d0e5-4151-9897-12c1c38bcb6a/send-request')


### switch to DM

In [22]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "Credential of Ownership")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange d27b4215-d0e5-4151-9897-12c1c38bcb6a, role: holder, state: credential_acked


## ask proof of ownership from SP

In [23]:
print("Request proof from SP")
#Set some variables

revocation = False
SELF_ATTESTED = True
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "possession", "restrictions": [{"issuer_did": dm_did}]},
    {"name": "time", "restrictions": [{"issuer_did": dm_did}]},
]

if revocation:
    req_attrs.append(
        {
            "name": "owner",
            "restrictions": [{"issuer_did": agent.did}],
            "non_revoked": {"to": int(time.time() - 1)},
        },
    )



#Set predicates for Zero Knowledge Proofs
req_preds = []

indy_proof_request = {
    "name": "Proof of Ownership",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": sp_connection,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

Request proof from SP


In [24]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

{'trace': False, 'presentation_request': {'name': 'Proof of Ownership', 'version': '1.0', 'requested_attributes': {'0_possession_uuid': {'name': 'possession', 'restrictions': [{'issuer_did': 'None'}]}, '0_time_uuid': {'name': 'time', 'restrictions': [{'issuer_did': 'None'}]}}, 'requested_predicates': {}, 'nonce': '824512507423231212313172'}, 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'f22aecef-9e4c-4022-b311-ed5c73a67276', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIE93bmVyc2hpcCIsICJ2ZXJzaW9uIjogIjEuMCIsICJyZXF1ZXN0ZWRfYXR0cmlidXRlcyI6IHsiMF9wb3NzZXNzaW9uX3V1aWQiOiB7Im5hbWUiOiAicG9zc2Vzc2lvbiIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIk5vbmUifV19LCAiMF90aW1lX3V1aWQiOiB7Im5hbWUiOiAidGltZSIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIk5vbmUifV19fSwgInJlcXVlc3RlZF9wcmVkaWNhdGVzIjoge30sICJub25jZSI6

### switch to SP

In [34]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)

{'auto_present': False, 'created_at': '2023-01-03 06:47:07.801744Z', 'thread_id': '4c0c7b44-6afd-4ef3-b3f1-facab14b608b', 'state': 'verified', 'updated_at': '2023-01-03 06:53:45.700838Z', 'presentation_request': {'name': 'Proof of Ownership', 'version': '1.0', 'requested_attributes': {'0_possession_uuid': {'name': 'possession', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_time_uuid': {'name': 'time', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'requested_predicates': {}, 'nonce': '84982767156097137759648'}, 'verified': 'true', 'role': 'verifier', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '4c0c7b44-6afd-4ef3-b3f1-facab14b608b', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIE93bmVyc2hpcCIsICJ2ZXJzaW9uIjogIjEuMCIsICJyZXF1ZXN0ZWRfYXR0cmlidXRlcyI6IHsiMF9wb3NzZXNzaW9uX3

In [35]:
print(verify['state'])
print(verify['state'] == 'verified')

verified
True


In [36]:
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(name + " : " + val['raw'])

0_possession_uuid : gZj39Y8JGQ5GVQp2y6zFH
0_time_uuid : 01-03-2023, 06:22:15


In [75]:
response = await agent_controller.terminate()
print(response)

None
